In [1]:
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions \
import load_preproc_data_adult #, load_preproc_data_german, load_preproc_data_compas

from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult #, get_distortion_german, get_distortion_compas

# Bias mitigation preproc-techniques

from aif360.sklearn.preprocessing import Reweighing

from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.algorithms.preprocessing.lfr import LFR

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
import load_preproc_data_adult

# from aif360.algorithms.preprocessing.optim_preproc_helpers import data_preproc_functions

from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

#from aif360.algorithms.inprocessing import PrejudiceRemover

# from aif360.datasets.lime_encoder import LimeEncoder
# import lime
# from lime.lime_tabular import LimeTabularExplainer

import pandas as pd
from sklearn.model_selection import train_test_split
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StructuredDataset
from aif360.datasets import AdultDataset
import numpy as np



# inprocessing
# from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
from aif360.sklearn.inprocessing import AdversarialDebiasing
import tensorflow as tf


# postprocessing
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
# Odds equalizing post-processing algorithm
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
# from aif360.algorithms.postprocessing.eq_odds_postprocessing import EqOddsPostprocessing

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [2]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])
dataset_orig.feature_names

['race',
 'sex',
 'Age (decade)=10',
 'Age (decade)=20',
 'Age (decade)=30',
 'Age (decade)=40',
 'Age (decade)=50',
 'Age (decade)=60',
 'Age (decade)=>=70',
 'Education Years=6',
 'Education Years=7',
 'Education Years=8',
 'Education Years=9',
 'Education Years=10',
 'Education Years=11',
 'Education Years=12',
 'Education Years=<6',
 'Education Years=>12']

In [3]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

In [4]:
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

In [5]:
protected = 'sex'
# dataset_orig_train.features = scaler.fit_transform(dataset_orig_train.features)
# dataset_orig_valid.features = scaler.fit_transform(dataset_orig_valid.features)
# dataset_orig_test.features = scaler.fit_transform(dataset_orig_test.features)

index = dataset_orig_train.feature_names.index(protected)

# di = DisparateImpactRemover(repair_level=1)
# data = di.fit_transform(dataset_orig_train)
# data

In [6]:
debug = 0

In [7]:
def pred_preproc_fit_model(train, valid, test):

    # fitting on training data
    scale_orig = StandardScaler()
    x_train = scale_orig.fit_transform(train.features)
    y_train = train.labels.ravel()
    # w_train = train.instance_weights.ravel()

    if debug==1:
        print("Scaler OK")

    lmod = LogisticRegression()
    lmod.fit(x_train, y_train,
         sample_weight=train.instance_weights)
    
    if debug==1:
        print("Model fit OK")
    

    # prediction on training data
    y_train_pred = lmod.predict(x_train)

    if debug==1:
        print("y pred OK")

    # positive class index
    pos_ind = np.where(lmod.classes_ == train.favorable_label)[0][0]

    if debug==1:
        print("pos_ind OK")

    train_pred = train.copy(deepcopy=True)
    # train_pred_labels = y_train_pred


    # prediction on validation and test data and score calculation
    valid_pred = valid.copy(deepcopy=True)
    X_valid = scale_orig.transform(valid_pred.features)
    # y_valid = valid_pred.labels
    valid_pred.scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

    if debug==1:
        print("valid OK")

    test_pred = test.copy(deepcopy=True)
    X_test = scale_orig.transform(test_pred.features)
    # y_test = test_pred.labels
    test_pred.scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

    if debug==1:
        print("test OK")

    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)

    if debug==1:
        print("def thresh OK")



    # balanced accuracy, threshold
    for idx, class_thresh in enumerate(class_thresh_arr):

        fav_inds = valid_pred.scores > class_thresh
        valid_pred.labels[fav_inds] = valid_pred.favorable_label
        valid_pred.labels[~fav_inds] = valid_pred.unfavorable_label

        if debug==1:
            print("best thresh sel OK")

        classified_metric_valid = ClassificationMetric(valid,
                                                 valid_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
        if debug==1:
            print("class metric valid OK")


        ba_arr[idx] = 0.5*(classified_metric_valid.true_positive_rate()\
                       +classified_metric_valid.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    if debug==1:
        print("best class thresh OK")

    

    return train_pred, valid_pred, test_pred, best_class_thresh

In [8]:
def reweighing(datas, unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups):

#     scale_orig = StandardScaler()
#     data= scale_orig.fit_transform(data)
    dataset_transf = []
    for data in datas:
      RW = Reweighing(unprivileged_groups=unprivileged_groups,
                    privileged_groups=privileged_groups)

      RW.fit(data)

      dataset_transf.append(RW.transform(data))

    return dataset_transf


def disparate_impact_remover(datas, level=1):

    # scale_orig = StandardScaler()
    # data.features= scale_orig.fit_transform(data.features)

    dataset_transf = []
    for data in datas:
      di = DisparateImpactRemover(repair_level=level)
      dataset_transf.append( di.fit_transform(data))

    return dataset_transf


def lfr(datas, unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups):

    # scale_orig = StandardScaler()
    # data.features = scale_orig.fit_transform(data.features)

    dataset_transf = []
    for data in datas:
      TR = LFR(unprivileged_groups=unprivileged_groups,
          privileged_groups=privileged_groups,
          k=10, Ax=0.1, Ay=1.0, Az=2.0,
          verbose=1
          )
      TR = TR.fit(data, maxiter=5000, maxfun=5000)
      dataset_transf.append(TR.transform(data))

    return dataset_transf



def optim_preproc(datas):

  dataset_transf = []
  for data in datas:
    optim_options = {
      "distortion_fun": get_distortion_adult,
      "epsilon": 0.05,
      "clist": [0.99, 1.99, 2.99],
      "dlist": [.1, 0.05, 0]
    }
        
    OP = OptimPreproc(OptTools, optim_options)
    OP = OP.fit(data)
    temp = OP.transform(data, transform_Y=True)
    dataset_transf.append(data.align_datasets(temp))

  return dataset_transf


# # From example 
# def adversial_debiasing(datas):
#   dataset_transf = []
#   tf.compat.v1.disable_eager_execution()
#   tf.logging.set_verbosity(tf.logging.ERROR)
  
#   for data in datas:
#     if np.any(data.features==None):
#       continue
#     # sess = tf.Session()
#     with tf.compat.v1.variable_scope('debiased_classifier', reuse=tf.compat.v1.AUTO_REUSE):
      
#       with tf.compat.v1.Session() as sess:
#         debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
#                               unprivileged_groups = unprivileged_groups,
#                               scope_name='debiased_classifier',
#                               debias=True,
#                               sess=sess)
#         sess.run(tf.compat.v1.global_variables_initializer())  # Initialize global variables
#         sess.run(tf.compat.v1.local_variables_initializer())
#         debiased_model.fit(data)


#         dataset_transf.append(debiased_model.predict(data))

#     sess.close()

#   return dataset_transf



# def adversial_debiasing(datas):
#   dataset_transf = []
  
#   for data in datas:
#     if np.any(data.features==None):
#       continue
#     # sess = tf.Session()
#     # with tf.compat.v1.variable_scope('debiased_classifier', reuse=tf.compat.v1.AUTO_REUSE):
#     tf.compat.v1.disable_eager_execution()
#       # with tf.compat.v1.Session() as sess:
#     debiased_model = AdversarialDebiasing(
#                                           prot_attr=[data.protected_attributes], 
#                                           # scope_name='debiased_classifier', 
#                                           # adversary_loss_weight=0.1, 
#                                           # num_epochs=50, batch_size=128, 
#                                           # classifier_num_hidden_units=200, 
#                                           debias=True, verbose=False, random_state=None)
#         # sess.run(tf.compat.v1.global_variables_initializer())  # Initialize global variables
#         # sess.run(tf.compat.v1.local_variables_initializer())
    
    
#     # print(type(data))
#     # df = StructuredDataset.convert_to_dataframe(data)[0]
#     # # print(df)

#     # x = df.drop(data.label_names, axis=1)
#     # y = df[data.label_names[0]]

#     # print(y)
   
#     # data = StructuredDataset(df, data.label_names, data.protected_attributes, data.instance_weights)

#     debiased_model.fit(data.features, data.labels.ravel())


#     dataset_transf.append(debiased_model.predict(data))

#     # sess.close()

#   return dataset_transf


def in1(ls):      # adversial debiasing
  return ls

def in2(ls):
  return ls

def in3(ls):
    return ls

def RejOpt(datas, datapreds):
  dataset_transf = []
  for data, data_pred in zip(datas, datapreds):
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                  privileged_groups=privileged_groups, 
                                  low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name="Statistical parity difference",
                                    metric_ub=0.05, metric_lb=-0.05)
    ROC = ROC.fit(data, data_pred)
    dataset_transf.append(ROC.predict(data))

    # dataset_transf.append(ROC.fit_transform(data, pred))

  return dataset_transf

def Cal_EqOdds(datas, datapreds):
  dataset_transf = []
  for data, data_pred in zip(datas,datapreds):
    cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint='weighted'
                                     )
    cpp = cpp.fit(data, data_pred)
    dataset_transf.append(cpp.predict(data_pred))
  
  return dataset_transf

''' 

        From Eqodds postprocessing library source file:

        metric = ClassificationMetric(dataset_true, dataset_pred,
            unprivileged_groups=self.unprivileged_groups,
            privileged_groups=self.privileged_groups)

        # compute basic statistics
        sbr = metric.base_rate(privileged=True)
        obr = metric.base_rate(privileged=False)

        fpr0 = metric.false_positive_rate(privileged=True)
        fpr1 = metric.false_positive_rate(privileged=False)
        fnr0 = metric.false_negative_rate(privileged=True)
        fnr1 = metric.false_negative_rate(privileged=False)
        tpr0 = metric.true_positive_rate(privileged=True)
        tpr1 = metric.true_positive_rate(privileged=False)
        tnr0 = metric.true_negative_rate(privileged=True)
        tnr1 = metric.true_negative_rate(privileged=False)

        # linear program has 4 decision variables:
        # [Pr[label_tilde = 1 | label_hat = 1, protected_attributes = 0];
        #  Pr[label_tilde = 1 | label_hat = 0, protected_attributes = 0];
        #  Pr[label_tilde = 1 | label_hat = 1, protected_attributes = 1];
        #  Pr[label_tilde = 1 | label_hat = 0, protected_attributes = 1]]
        # Coefficients of the linear objective function to be minimized.
        c = np.array([fpr0 - tpr0, tnr0 - fnr0, fpr1 - tpr1, tnr1 - fnr1])
        

        Error on executing below function
        ValueError: Invalid input for linprog: c must not contain values inf, nan, or None
        '''

# def EqOdds(datas, datapreds):
#   dataset_transf = []
#   for data, data_pred in zip(datas,datapreds):
#     epp = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=None)
#     epp = epp.fit(data, data_pred)
#     dataset_transf.append(epp.predict(data_pred))

#   return dataset_transf


def EqOdds(datas, datapreds):
  return datapreds

def evaluate_performance(post_processed_data, labels):
  pass

# empty function for original data
def pre_n_in_no_fn(ls):
  # train = ls[0]
  # valid = ls[1]
  # test = ls[2]
  # return train,valid,test
  return ls

def post_no_fn(ls, ls2):
  # train = ls[0]
  # valid = ls[1]
  # test = ls[2]
  # return train,valid,test
  return ls2

In [9]:
# retest rejopt function with fit_transform

if debug == 1:
    train_pred, valid_pred, test_pred, best_class_thresh = pred_preproc_fit_model(dataset_orig_train, dataset_orig_valid, dataset_orig_test)
    ls = EqOdds([dataset_orig_train, dataset_orig_valid, dataset_orig_test],[train_pred, valid_pred, test_pred])
    print(ls)

In [10]:
# if debug == 1:
#     ls = adversial_debiasing([dataset_orig_train, dataset_orig_valid, dataset_orig_test])
#     ls

In [11]:
dataset_orig_train.features

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [12]:
def cal_metric(test, split_data_pred, best_class_thresh):
        fav_inds = split_data_pred.scores > best_class_thresh
        split_data_pred.labels[fav_inds] = split_data_pred.favorable_label
        split_data_pred.labels[~fav_inds] = split_data_pred.unfavorable_label

        

        metric_obj = ClassificationMetric(test, split_data_pred,
                                        unprivileged_groups, privileged_groups)
        
        metric = []

        metric.append(abs(metric_obj.mean_difference()))
        metric.append(abs(1-metric_obj.disparate_impact()))
        # metric.append(abs(metric_obj.equal_opportunity_difference()))
        # metric.append(abs(metric_obj.average_odds_difference()))
        metric.append(abs(metric_obj.theil_index()))

        avg = 0
        for m in metric:
            avg += m

        avg = avg/len(metric)

        return avg

In [13]:
from copy import deepcopy
from itertools import product

def test_preprocessing_combinations(train,valid,test):
    # Define the preprocessing algorithms
    preprocessing_algos = {
        'pre1': reweighing,
        'pre2': disparate_impact_remover,
        'pre3': lfr,
        'pre4': optim_preproc,                           #### try later
        'pre_none': pre_n_in_no_fn
    }

    # Define the in-processing algorithms
    in_processing_algos = {
        'in1': in1,
        'in2': in2,
        'in3': in3,
        'in_none': pre_n_in_no_fn
    }

    # Define the post-processing algorithms
    post_processing_algos = {
        'post1': RejOpt,
        'post2': Cal_EqOdds,
        'post3': EqOdds,
        'post_none': post_no_fn
    }

    # Store the scores for each combination
    combination_scores = {}

    # Generate all possible combinations of preprocessing, in-processing, and post-processing algorithms
    all_combinations = product(preprocessing_algos.keys(), 
                               #in_processing_algos.keys(), 
                               post_processing_algos.keys())

    preproc_data = {}

    # Test each combination and store the scores
    for combination in all_combinations:
        print(combination)
        # pre_algo, in_algo, post_algo = combination
        pre_algo, post_algo = combination

        # train = og_train.copy(deepcopy=True)
        # valid = og_valid.copy(deepcopy=True)
        # test = og_test.copy(deepcopy=True)


        try:
        #   preprocessed_data = preproc_data[pre_algo]
            ls = preproc_data[pre_algo]
            # pre_train = ls[0]
            # pre_valid = ls[1]
            # pre_test = ls[2]

            # optimizing space comp
            trf_train = ls[0]
            trf_valid = ls[1]
            trf_test = ls[2]

        except:
          # Apply the preprocessing algorithm
          # preprocessed_data = preprocessing_algos[pre_algo](train, valid, test)
            [trf_train, trf_valid, trf_test] = preprocessing_algos[pre_algo]([train, valid, test])
            # preproc_data[pre_algo] = preprocessed_data
            preproc_data[pre_algo] = [trf_train, trf_valid, trf_test]
        
        if debug ==1:
            # if og_train == train:
            #     print("train not modif")
            print(trf_train, trf_valid, trf_test)
            # check_pt = pre_train.copy(deepcopy=True)
            # check_pv = pre_valid.copy(deepcopy=True)
            # check_ptt = pre_test.copy(deepcopy=True)

        

        train_pred, valid_pred, test_pred, best_class_thresh = pred_preproc_fit_model(trf_train, trf_valid, trf_test)

        if debug==1:
            # if check_pt == pre_train:
                # print("pre train not modif")
            print("pred proc OK")

            # print(train_pred, valid_pred, test_pred, best_class_thresh)
            # print(train_pred)
            # print(valid_pred)
            print("test: \n", test)
            print("test_pred: \n", test_pred)
            # print(best_class_thresh)
        
        # Apply the in-processing algorithm
        # [trf_train, trf_valid, trf_test] = in_processing_algos[in_algo]([trf_train, trf_valid, trf_test])

        # Apply the post-processing algorithm
        [trf_train, trf_valid, trf_test] = post_processing_algos[post_algo]([trf_train, trf_valid, trf_test], [train_pred, valid_pred, test_pred])

        # Evaluate the performance and store the score
        # score = evaluate_performance(post_processed_data, labels)
        
        # # removing labels from test pred for metric function
        # temp_label = test_pred.labels
        # del test_pred.labels
        # # test_pred.labels = None
        # test2 = test_pred.copy(deepcopy=True)
        # test_pred.labels = temp_label

        # # removing labels try 2
        # temp_label = test_pred.labels
        # del test_pred.features['labels']  # Remove the 'labels' column from the features
        # test2 = test_pred.copy(deepcopy=True)
        # # Assign the original labels back to the modified dataset
        # test_pred.features['labels'] = temp_label

        # test_pred = BinaryLabelDataset(
        #     favorable_label=1,
        #     unfavorable_label=0,
        #     df=test_pred,
        #     label_names = test_pred.labels,
        #     protected_attribute_names = ['sex']
        # )

        # split_data_pred = BinaryLabelDataset(
        #     favorable_label=1,
        #     unfavorable_label=0,
        #     df=split_data_pred,
        #     label_names = split_data_pred.labels,
        #     protected_attribute_names = ['sex']
        # )


        score = cal_metric(trf_test, test_pred, best_class_thresh)  
        combination_scores[combination] = score
        

    # Select the combination with the best score --> 0
    best_combination = min(combination_scores, key=combination_scores.get)
    best_score = combination_scores[best_combination]

    return best_combination, best_score


In [14]:
bc,bs = test_preprocessing_combinations(dataset_orig_train, dataset_orig_valid, dataset_orig_test)

('pre1', 'post1')
('pre1', 'post2')
('pre1', 'post3')
('pre1', 'post_none')
('pre2', 'post1')


c:\Users\singh\AppData\Local\Programs\Python\Python38\lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


('pre2', 'post2')
('pre2', 'post3')
('pre2', 'post_none')
('pre3', 'post1')
step: 0, loss: 0.8636315014491124, L_x: 0.49642392851890066,  L_y: 0.7931940303732408,  L_z: 0.010397539111990742
step: 250, loss: 0.7029605642414505, L_x: 0.49427718898149353,  L_y: 0.6333684704199326,  L_z: 0.010082187461684282
step: 500, loss: 0.6243306271728277, L_x: 0.4882211857772604,  L_y: 0.558106201952459,  L_z: 0.008701153321321358
step: 750, loss: 0.6163753111118245, L_x: 0.48396111789932506,  L_y: 0.5536038401466898,  L_z: 0.007187679587601102
step: 1000, loss: 0.6000747383908153, L_x: 0.45804504459421524,  L_y: 0.5434536323011926,  L_z: 0.005408300815100571
step: 1250, loss: 0.5898288482950883, L_x: 0.4309105060096078,  L_y: 0.5351520294232156,  L_z: 0.00579288413545598
step: 1500, loss: 0.577620337038947, L_x: 0.39823017631517976,  L_y: 0.5279737863462755,  L_z: 0.004911766530576815
step: 1750, loss: 0.5460172340162424, L_x: 0.28503508611712725,  L_y: 0.49824343507660723,  L_z: 0.00963514516396121

c:\Users\singh\AppData\Local\Programs\Python\Python38\lib\site-packages\aif360\metrics\dataset_metric.py:82: RuntimeWarning: invalid value encountered in double_scalars
  return metric_fun(privileged=False) / metric_fun(privileged=True)
c:\Users\singh\AppData\Local\Programs\Python\Python38\lib\site-packages\aif360\metrics\classification_metric.py:694: RuntimeWarning: invalid value encountered in divide
  return np.mean(np.log((b / np.mean(b))**b) / np.mean(b))


('pre3', 'post2')


...

Optimized Preprocessing: Objective converged to 0.012263


c:\Users\singh\AppData\Local\Programs\Python\Python38\lib\site-packages\aif360\algorithms\preprocessing\optim_preproc.py:261: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df2.loc[:, dfMap.columns.names] = [draws_possible[x] for x in draw_inds]
c:\Users\singh\AppData\Local\Programs\Python\Python38\lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


('pre4', 'post2')
('pre4', 'post3')
('pre4', 'post_none')
('pre_none', 'post1')


c:\Users\singh\AppData\Local\Programs\Python\Python38\lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


('pre_none', 'post2')
('pre_none', 'post3')
('pre_none', 'post_none')


In [15]:
bc

('pre1', 'post3')

In [16]:
bs

0.012884583961401685